In [1]:
import os
# os.getcwd()
os.chdir("/localdisk4/panwla/Projects/park_vlm/Kangaroo")
os.getcwd()

'/localdisk4/panwla/Projects/park_vlm/Kangaroo'

In [2]:
import json
import os
import boto3
import yaml
# from google.colab import drive

# # Mount Google Drive to access API keys
# drive.mount('/content/drive', force_remount=True)

# Load API keys from file
file_path = './.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)

### WANDB Keys
wandb_key = api_keys['WANDB']['Key']
hf_read_api_key = api_keys['HUGGINGFACE']['HF_READ_API_KEY']

# Extract AWS credentials
aws_access_key_id = api_keys['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = api_keys['AWS']['AWS_SECRET_ACCESS_KEY']

from huggingface_hub import login
login(hf_read_api_key)

# Initialize the S3 client with credentials
s3 = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

/localdisk4/panwla/conda_envs/env3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# import wandb
# wandb.login(key=wandb_key)

In [5]:

from huggingface_hub import login
login(hf_read_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [6]:
import os

# Set environment variables based on your bashrc settings
os.environ["JUPYTER_DATA_DIR"] = "/localdisk4/panwla/jupyter_data"
os.environ["HF_HOME"] = "/localdisk4/panwla/huggingface_cache"
os.environ["TRANSFORMERS_CACHE"] = "/localdisk4/panwla/huggingface_cache/transformers"
os.environ["HF_DATASETS_CACHE"] = "/localdisk4/panwla/huggingface_cache/datasets"
os.environ["TORCH_HOME"] = "/localdisk4/panwla/torch"
os.environ["XDG_CACHE_HOME"] = "/localdisk4/panwla_cache/.cache"
os.environ["TMPDIR"] = "/localdisk4/panwla/tmp"
os.environ["TEMP"] = "/localdisk4/panwla/tmp"
os.environ["TMP"] = "/localdisk4/panwla/tmp"

# Create directories if they don't exist
directories = [
    os.environ["JUPYTER_DATA_DIR"],
    os.environ["HF_HOME"],
    os.environ["TRANSFORMERS_CACHE"],
    os.environ["HF_DATASETS_CACHE"],
    os.environ["TORCH_HOME"],
    os.environ["XDG_CACHE_HOME"],
    os.environ["TMPDIR"],
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)


In [7]:
# !export CUDA_VISIBLE_DEVICES=0


In [ ]:
import av
import numpy as np
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration

def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


model = VideoLlavaForConditionalGeneration.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

prompt = "USER: <video> Why is this video funny? ASSISTANT:"
video_path = "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
container = av.open(video_path)

# sample uniformly 8 frames from the video
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip = read_video_pyav(container, indices)

inputs = processor(text=prompt, videos=clip, return_tensors="pt")

# Generate
generate_ids = model.generate(**inputs, max_length=80)
print(processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])


In [ ]:
from transformers import pipeline

# Try this instead
pipe = pipeline("visual-question-answering", model="liuhaotian/llava-v1.5-7b")

KeyboardInterrupt: 

In [9]:
def analyze_video(video_path, query):
    
    # Check if video file exists
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")
    
    # Define terminators (constant across all calls)
    terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]
    
    # Run the model's chat function with fixed parameters
    output, _ = model.chat(
        video_path=video_path,
        query=query,
        tokenizer=tokenizer,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    return output

facial_expressions_query = '''
Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Please describe whether the person demonstrates any difficulty through their facial expressions. Some examples of visible difficulty include furrowed brow, squinting eyes, clenched jaw, tight lips, head hanging low, sighing, wrinkled forehead, etc. Mention such specific details when found. End output with a final answer choice: "Yes" or "No".

Answer:
'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    facial_expressions_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


In [10]:
print("praveen kumar")

praveen kumar


In [12]:
facial_expressions_query = '''

Question: Can you tell me duration of the video?

Answer:
'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/File1.mp4", 
    facial_expressions_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 The video is 8 seconds long.


In [11]:

BG_and_lighting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Mention if the background is overloaded (i.e., too many things), or the lighting condition is inappropriate (i.e., too dark or overlit). Otherwise, just output “normal background”.

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    BG_and_lighting_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Normal background.


In [12]:

Blink_rate_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: Was there anything abnormal about the person’s eye blink rate? For example, they may not be blinking at all, or they may have reduced or increased blink rate compared to a normal person. If there is nothing abnormal, output “normal blinking”.

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Blink_rate_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Normal blinking.


In [13]:
Camera_position_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Answer the question about what is happening in the video.

Question: How far is the camera? A good position of the camera would be when the upper half of the subject's body remains visible, while the lower half is not captured in the frame. If the upper body is only partly visible, the camera is too close. If the lower body is also visible, the camera is too far.

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Camera_position_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 The camera is at a good distance.


In [14]:
Coherence_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Is the subject coherent in what they are speaking? Are they delivering an easy to understand story, or are they deterring a lot from the central topic and delivering an unorganized speech?

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Coherence_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes, the subject is coherent in what they are speaking. They are delivering an easy-to-understand story without any major detours from the central topic.


In [15]:
Comp_task_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>

Question: Please indicate whether the subject was able to follow the instructions while completing the task. If the subject was doing something differently, please describe.

Answer: 


'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Comp_task_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes, the subject was able to follow the instructions while completing the task.


In [16]:
Lip_parting_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate the extent of lips parting when the subject is not saying anything (i.e., always/most of the times/sometimes/very few times/never).

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Lip_parting_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Always.


In [17]:
Masked_faceExp_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate which of the following are true for the subject: (i) The individual's face appears blank and emotionless, even when they are trying to express an emotion. (ii) The expression is weak or asymmetrical, and the individual has difficulty holding an expression (e.g., smile) for an extended period. Also mention if you observe other facial expression abnormalities.

Answer: 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Masked_faceExp_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 (I) The individual's face appears blank and emotionless, even when they are trying to express an emotion


In [18]:
Observations_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Document any abnormal signs observed in body parts other than the face. This includes, but is not limited to, tremors in the hands, involuntary shaking or rhythmic movements of the upper or lower extremities, stiffness or rigidity in the limbs, reduced arm swing while speaking, or any signs of bradykinesia (slowness of movement). Additionally, note any abnormal postures, difficulty in maintaining balance, or other motor irregularities that may be indicative of Parkinson’s disease.

Answer: 


'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Observations_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 The video depicts an individual seated against a plain wall, engaged in a series of actions that are characteristic of a Parkinson's disease assessment. The individual, wearing a floral shirt and eyeglasses, appears to be participating in a speech task, speaking about a book or movie, and demonstrating other motor skills. The video captures the individual's facial expressions, body language, and any potential motor irregularities that may be indicative of Parkinson's disease.


In [19]:
Overall_app_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Provide a brief description of the subject's perceived state of mind, noting whether they appear energetic, exhausted, calm, confused, or exhibit any other relevant emotional or cognitive cues.

Answer: 


'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Overall_app_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 The subject appears to be in a state of confusion or frustration, as evidenced by their facial expressions and gestures. They may be struggling to articulate their thoughts or feelings, which could be indicative of a cognitive disorder or other mental health issue.


In [20]:
Other_people_present_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate whether any other individuals were present in the background. If so, provide a brief description (e.g., "An older male is visible in the background"). Conclude with a final answer: "Yes" or "No".

Answer: 
 

'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Other_people_present_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 No.


In [21]:
Comp_sent_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease.

Analyze the provided text transcription of the person’s speech and answer the question about what is happening in the video.

Transcription: <TRANSCRIPTION_OUTPUT>


Question: Indicate whether the subject is using complex sentences that are difficult to understand. Conclude with a final answer: "Yes" or "No".

Answer: 


'''

response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Comp_sent_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 Yes.


In [22]:
Other_BP_visible_query = '''

Imagine you are a clinician specializing in movement disorders. Rely on your knowledge of neurology and clinical care. Now, you are watching a home-recorded video of a person performing some tasks used to assess Parkinson's disease. No experts supervise the person, so there can be different types of noise, or the person may not follow the task instructions properly. The person can also show symptoms that may be associated with having Parkinson's disease. Focus on the noises, task instructions, user compliance, and possible symptoms of Parkinson's disease while answering the question.

Task instructions: The person will talk about a recent book they have read or a movie or TV show they have watched. The person will speak for approximately one minute. They should be front-facing the camera, and their face must be visible in the recording frame. There should not be any other person visible in the recording frame. The background should not be dark or overlit and should have good contrast against the person's face. For this task, the face is the most crucial body part you should focus on. However, you should also observe other body parts for relevant symptoms or signs of Parkinson's disease. 

Answer the question about what is happening in the video.


Question: Indicate if any body part critical for this task is partially visible (or invisible). For example, was the subject wearing a mask that may have obstructed important visual information? Or did the face go out of frame while the subject was completing the task?

Answer: 


'''


response = analyze_video(
    "/localdisk4/panwla/Projects/park_vlm/Kangaroo/ViViT/ViViT/resized_File1.mp4", 
    Other_BP_visible_query
)
print("Assistant:\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant:
 No, the subject's face is visible in the video, and no other part of their body is partially visible.
